In [416]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from matplotlib import pyplot as plt
from keras.models import Model
from sklearn.decomposition import PCA
import random
import keras
from keras import backend as K
import pandas as pd
from sklearn.manifold import TSNE
from keras.models import Sequential
from keras.layers.core import Dense,Activation
import numpy as geek
import soundfile as sf
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv1D, Dense, MaxPooling1D, Flatten,Conv2D,MaxPooling2D
from tensorflow.keras.layers import BatchNormalization, Dropout
from IPython.display import Audio

## **Part 1 DNN Model**

**Importing the dataset clean and dirty**

In [373]:
import librosa
s, sr = librosa.load('train_clean_male.wav',sr=None)
S = librosa.stft(s, n_fft=1024, hop_length=512)
sn, sr = librosa.load('train_dirty_male.wav',sr=None)
X = librosa.stft(sn, n_fft =1024,hop_length=512)

In [374]:
S.shape, X.shape

((513, 2459), (513, 2459))

In [248]:
S

array([[ 1.1937854e-02+0.00000000e+00j,  1.1326679e-02+0.00000000e+00j,
         1.1311449e-02+0.00000000e+00j, ...,
        -1.9435661e-02+0.00000000e+00j, -8.2834195e-03+0.00000000e+00j,
         2.1649147e-03+0.00000000e+00j],
       [-7.4651041e-03-1.32137140e-19j, -5.8471980e-03+1.92323478e-03j,
        -4.9916869e-03+1.33694301e-03j, ...,
         1.2399258e-02-2.94200727e-03j,  3.2734389e-03+1.41162460e-03j,
         4.6770875e-03-1.11858710e-03j],
       [ 2.1543016e-03+2.88838235e-19j, -3.9865682e-04-1.69004453e-03j,
        -4.4653739e-04-3.15565290e-03j, ...,
        -3.0208891e-03-6.29775343e-04j,  3.8380092e-03+1.11574605e-02j,
        -9.1405194e-03+8.48239847e-03j],
       ...,
       [ 3.9365393e-04-1.80418018e-19j, -7.1495574e-06+1.27830135e-03j,
         8.8752218e-04-3.28958355e-04j, ...,
        -8.9677225e-05-9.15538869e-04j, -6.1033864e-04-5.72167337e-04j,
        -4.1909068e-04-1.59045146e-03j],
       [-5.0875830e-04+4.03187683e-19j, -4.1133209e-04-1.02382002e-0

In [375]:
S_ab = np.abs(S)
X_ab = np.abs(X)

In [250]:
S_ab

array([[0.01193785, 0.01132668, 0.01131145, ..., 0.01943566, 0.00828342,
        0.00216491],
       [0.0074651 , 0.00615537, 0.00516763, ..., 0.01274351, 0.00356484,
        0.00480899],
       [0.0021543 , 0.00173643, 0.00318709, ..., 0.00308584, 0.01179912,
        0.01246997],
       ...,
       [0.00039365, 0.00127832, 0.00094652, ..., 0.00091992, 0.00083659,
        0.00164474],
       [0.00050876, 0.00110336, 0.00104441, ..., 0.00122437, 0.00183458,
        0.00079415],
       [0.00026081, 0.00068581, 0.0013387 , ..., 0.00146104, 0.00243181,
        0.00176089]], dtype=float32)

In [377]:
S_ab_T = S_ab.T
X_ab_T = X_ab.T
X_ab_T.shape,S_ab_T.shape

((2459, 513), (2459, 513))

**Spliiting the data into train set and validation set**

In [252]:
X_ab_T_train = X_ab_T[0:2000,]
X_ab_T_val = X_ab_T[2000:,]
S_ab_T_train = S_ab_T[0:2000,]
S_ab_T_val = S_ab_T[2000:,]

**Creating and Running the baseline DNN model**

In [228]:
initializer = tf.keras.initializers.HeNormal()
baseline = Sequential()
baseline.add(Dense(1024, input_dim=513,kernel_initializer=initializer))
baseline.add(Activation('relu'))
baseline.add(Dense(1024))
baseline.add(Activation('relu'))
baseline.add(Dense(513))
baseline.add(Activation('relu'))

In [229]:
baseline.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_48 (Dense)             (None, 1024)              526336    
_________________________________________________________________
activation_72 (Activation)   (None, 1024)              0         
_________________________________________________________________
dense_49 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
activation_73 (Activation)   (None, 1024)              0         
_________________________________________________________________
dense_50 (Dense)             (None, 513)               525825    
_________________________________________________________________
activation_74 (Activation)   (None, 513)               0         
Total params: 2,101,761
Trainable params: 2,101,761
Non-trainable params: 0
___________________________________________

In [230]:
optimize_adam = tf.keras.optimizers.Adam(learning_rate=0.001)
baseline.compile(optimizer=optimize_adam,loss=tf.keras.losses.MeanSquaredError(),metrics=['accuracy'])

In [231]:
#Ref: https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/
es = [EarlyStopping(monitor='val_loss',patience=200,verbose=1,mode='auto')]
baseline.fit(X_ab_T_train,S_ab_T_train,epochs=500,batch_size=32,validation_data=(X_ab_T_val,S_ab_T_val),callbacks=es)

Epoch 1/500
63/63 [==============================] - 1s 4ms/step - loss: 0.0391 - accuracy: 0.2250 - val_loss: 0.0301 - val_accuracy: 0.3333
Epoch 2/500
63/63 [==============================] - 0s 2ms/step - loss: 0.0198 - accuracy: 0.3595 - val_loss: 0.0257 - val_accuracy: 0.3791
Epoch 3/500
63/63 [==============================] - 0s 2ms/step - loss: 0.0170 - accuracy: 0.3880 - val_loss: 0.0241 - val_accuracy: 0.3834
Epoch 4/500
63/63 [==============================] - 0s 2ms/step - loss: 0.0154 - accuracy: 0.3950 - val_loss: 0.0225 - val_accuracy: 0.3943
Epoch 5/500
63/63 [==============================] - 0s 2ms/step - loss: 0.0148 - accuracy: 0.4030 - val_loss: 0.0211 - val_accuracy: 0.3965
Epoch 6/500
63/63 [==============================] - 0s 2ms/step - loss: 0.0143 - accuracy: 0.4170 - val_loss: 0.0214 - val_accuracy: 0.3834
Epoch 7/500
63/63 [==============================] - 0s 2ms/step - loss: 0.0134 - accuracy: 0.4180 - val_loss: 0.0209 - val_accuracy: 0.4031
Epoch 8/500
6

**Testing the output of the DNN model on test set 1**

In [235]:
s, sr = librosa.load('test_x_01.wav',sr=None)
Xtest = librosa.stft(s, n_fft=1024, hop_length=512)
Xtest_ab = np.abs(Xtest)
Xtest_ab.shape

(513, 142)

In [236]:
Stest_ab = baseline.predict(Xtest_ab.T)
Stest_ab.shape

(142, 513)

In [237]:
Stest_ab = Stest_ab.T
Stest_ab.shape

(513, 142)

In [238]:
Stest = geek.multiply(np.divide(Xtest,Xtest_ab), Stest_ab)
Stest_inv = librosa.istft(Stest,hop_length=512)
sf.write('test_s_01_recons.wav',Stest_inv,sr)

In [421]:
sound_file = 'test_s_01_recons-DNN.wav'
Audio(sound_file, autoplay=True)

**Testing the output of the DNN model on test set 2**

In [239]:
s, sr = librosa.load('test_x_02.wav',sr=None)
Xtest = librosa.stft(s, n_fft=1024, hop_length=512)
Xtest_ab = np.abs(Xtest)
Xtest_ab.shape

(513, 380)

In [240]:
Stest_ab = baseline.predict(Xtest_ab.T)
Stest_ab.shape

(380, 513)

In [241]:
Stest_ab = Stest_ab.T
Stest_ab.shape

(513, 380)

In [242]:
Stest = geek.multiply(np.divide(Xtest,Xtest_ab), Stest_ab) 
Stest.shape

(513, 380)

In [243]:
Stest_inv = librosa.istft(Stest,hop_length=512)
Stest_inv.shape
sf.write('test_s_02_recons.wav',Stest_inv,sr)

In [422]:
sound_file = 'test_s_02_recons-DNN.wav'
Audio(sound_file, autoplay=True)

**Checking the Signal to Noise Ratio for DNN model**

In [244]:
S_ab_T_pred = baseline.predict(X_ab_T_val)

In [245]:
def SNR_func(y,ypred):
  num = np.sum(np.square(y))
  dem = np.sum(np.square(y-ypred))
  return 10 * np.log10(num/dem)

In [246]:
SNR_func(S_ab_T_val,S_ab_T_pred)

8.095734715461731

## **Part 2 CNN 1D**

In [378]:
X_ab_T.shape

(2459, 513)

In [379]:
X_ab_T = X_ab_T.reshape(2459,513,1)

In [380]:
S_ab_T.shape

(2459, 513)

**Spliiting the data into train set and validation set**

In [381]:
X_ab_T_train = X_ab_T[0:2200,]
X_ab_T_val = X_ab_T[2200:,] 
S_ab_T_train = S_ab_T[0:2200,]
S_ab_T_val = S_ab_T[2200:,]

In [382]:
X_ab_T_train.shape,X_ab_T_val.shape

((2200, 513, 1), (259, 513, 1))

**Creating and Running the CNN 1D model**

In [395]:
initializer = tf.keras.initializers.HeNormal()
model1d = Sequential()
model1d.add(Conv1D(filters=32,kernel_size=4,input_shape=(513,1),padding='valid',kernel_initializer=initializer))
model1d.add(MaxPooling1D(pool_size=2,strides=2))
model1d.add(Activation('relu'))
model1d.add(BatchNormalization())
model1d.add(Conv1D(filters=16,kernel_size=2,padding='valid'))
model1d.add(MaxPooling1D(pool_size=2,strides=2))
model1d.add(Activation('relu'))
model1d.add(BatchNormalization())
model1d.add(Flatten())
model1d.add(Dense(1024))
model1d.add(Activation('relu'))
model1d.add(BatchNormalization())
model1d.add(Dense(513))
model1d.add(Activation('relu'))

In [396]:
model1d.summary()

Model: "sequential_37"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_22 (Conv1D)           (None, 510, 32)           160       
_________________________________________________________________
max_pooling1d_22 (MaxPooling (None, 255, 32)           0         
_________________________________________________________________
activation_111 (Activation)  (None, 255, 32)           0         
_________________________________________________________________
batch_normalization_56 (Batc (None, 255, 32)           128       
_________________________________________________________________
conv1d_23 (Conv1D)           (None, 254, 16)           1040      
_________________________________________________________________
max_pooling1d_23 (MaxPooling (None, 127, 16)           0         
_________________________________________________________________
activation_112 (Activation)  (None, 127, 16)         

In [398]:
optimize_adam = tf.keras.optimizers.Adam(learning_rate=0.0001)
model1d.compile(optimizer=optimize_adam,loss=tf.keras.losses.MeanSquaredError(),metrics=['accuracy'])

In [399]:
#Ref: https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/
model1d.fit(X_ab_T_train,S_ab_T_train,epochs=200,batch_size=32,validation_data=(X_ab_T_val,S_ab_T_val))

Epoch 1/200
69/69 [==============================] - 1s 7ms/step - loss: 0.2958 - accuracy: 0.0177 - val_loss: 0.0507 - val_accuracy: 0.0618
Epoch 2/200
69/69 [==============================] - 0s 5ms/step - loss: 0.1084 - accuracy: 0.0845 - val_loss: 0.0451 - val_accuracy: 0.1042
Epoch 3/200
69/69 [==============================] - 0s 4ms/step - loss: 0.0718 - accuracy: 0.1264 - val_loss: 0.0387 - val_accuracy: 0.1236
Epoch 4/200
69/69 [==============================] - 0s 4ms/step - loss: 0.0565 - accuracy: 0.1632 - val_loss: 0.0342 - val_accuracy: 0.1506
Epoch 5/200
69/69 [==============================] - 0s 4ms/step - loss: 0.0476 - accuracy: 0.1841 - val_loss: 0.0305 - val_accuracy: 0.1969
Epoch 6/200
69/69 [==============================] - 0s 4ms/step - loss: 0.0396 - accuracy: 0.2045 - val_loss: 0.0278 - val_accuracy: 0.1699
Epoch 7/200
69/69 [==============================] - 0s 4ms/step - loss: 0.0345 - accuracy: 0.2155 - val_loss: 0.0252 - val_accuracy: 0.2162
Epoch 8/200
6

**Testing the output of the CNN 1D model on test set 1**

In [400]:
s, sr = librosa.load('test_x_01.wav',sr=None)
Xtest = librosa.stft(s, n_fft=1024, hop_length=512)
Xtest_ab = np.abs(Xtest)
Xtest_ab_T = Xtest_ab.T
Xtest_ab_T.shape

(142, 513)

In [401]:
Xtest_ab_T = Xtest_ab_T.reshape(142,513,1)
Stest_ab = model1d.predict(Xtest_ab_T)
Stest_ab.shape

(142, 513)

In [402]:
Stest_ab = Stest_ab.T
Stest_ab.shape

(513, 142)

In [404]:
Stest = geek.multiply(np.divide(Xtest,Xtest_ab), Stest_ab)
Stest_inv = librosa.istft(Stest,hop_length=512)
sf.write('test_s_01_recons.wav',Stest_inv,sr)

In [427]:
sound_file = 'test_s_01_recons_CNN_1D.wav'
Audio(sound_file, autoplay=True)

**Testing the output of the CNN 1D model on test set 2**

In [335]:
s, sr = librosa.load('test_x_02.wav',sr=None)
Xtest = librosa.stft(s, n_fft=1024, hop_length=512)
Xtest_ab = np.abs(Xtest)
Xtest_ab.shape

(513, 380)

In [336]:
Xtest_ab_T = Xtest_ab.T.reshape(380,513,1)
Stest_ab =  model1d.predict(Xtest_ab_T)
Stest_ab.shape

(380, 513)

In [337]:
Stest_ab = Stest_ab.T
Stest_ab.shape

(513, 380)

In [338]:
Stest = geek.multiply(np.divide(Xtest,Xtest_ab), Stest_ab) 
Stest.shape

(513, 380)

In [339]:
Stest_inv = librosa.istft(Stest,hop_length=512)
Stest_inv.shape
sf.write('test_s_02_recons.wav',Stest_inv,sr)

In [429]:
sound_file = 'test_s_02_recons-CNN_1D.wav'
Audio(sound_file, autoplay=True)

**Checking the Signal to Noise Ratio for CNN 1D model**

In [405]:
S_ab_T_pred = model1d.predict(X_ab_T_val)

In [406]:
def SNR_func(y,ypred):
  num = np.sum(np.square(y))
  dem = np.sum(np.square(y-ypred))
  return 10 * np.log10(num/dem)

In [409]:
SNR_func(S_ab_T_val,S_ab_T_pred)

10.1237890453278


## **Part 3 CNN 2D Model**

In [99]:
S_ab = np.abs(S)
X_ab = np.abs(X)

In [100]:
silent_frames = np.random.normal(0,0.00002,size=(19, 20, 513)) +.00000001
silent_frames.shape

(19, 20, 513)

**Creating images from the audio input by having 20 X512 magnitude spectograms. Each iamge will be 20 frames shifted by one frame**

In [101]:
def img_build(input):
  img_add = []
  for i in range(input.shape[0]-19):
    img_add.append(input[i:i+20,0:513])
  img_add = np.array(img_add)
  img_add = np.concatenate((silent_frames,img_add))
  return img_add

In [102]:
X_ab.shape,S_ab.shape

((513, 2459), (513, 2459))

In [103]:
X_ab_Img = img_build(X_ab.T)
X_ab_Img.shape

(2459, 20, 513)

In [104]:
X_ab_Img = X_ab_Img.reshape(2459,20,513,1)

**Spliiting the data into train set and validation set**

In [109]:
X_ab_Img_train = X_ab_Img[0:2200,]
X_ab_Img_val = X_ab_Img[2200:,]
S_ab_Img_train = S_ab.T[0:2200,]
S_ab_Img_val = S_ab.T[2200:,]

In [110]:
X_ab_Img_train.shape, S_ab_Img_train.shape

((2200, 20, 513, 1), (2200, 513))

In [111]:
X_ab_Img_val.shape, S_ab_Img_val.shape

((259, 20, 513, 1), (259, 513))

**Creating and Running the CNN 2D model**

In [198]:
initializer = tf.keras.initializers.GlorotNormal()
model2d = Sequential()
model2d.add(Conv2D(filters=16,kernel_size=(7,7),padding = 'valid',input_shape=(20,513,1),kernel_initializer=initializer))
model2d.add(MaxPooling2D(pool_size=(2,2)))
model2d.add(Activation('relu'))
model2d.add(BatchNormalization())
model2d.add(Flatten())
model2d.add(Dense(1024))
model2d.add(Activation('relu'))
model2d.add(BatchNormalization())
model2d.add(Dense(513))
model2d.add(Activation('relu'))

In [199]:
model2d.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_23 (Conv2D)           (None, 14, 507, 16)       800       
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 7, 253, 16)        0         
_________________________________________________________________
activation_66 (Activation)   (None, 7, 253, 16)        0         
_________________________________________________________________
batch_normalization_40 (Batc (None, 7, 253, 16)        64        
_________________________________________________________________
flatten_22 (Flatten)         (None, 28336)             0         
_________________________________________________________________
dense_43 (Dense)             (None, 1024)              29017088  
_________________________________________________________________
activation_67 (Activation)   (None, 1024)            

In [200]:
optimize_adam = tf.keras.optimizers.Adam(0.0001)
model2d.compile(optimizer=optimize_adam,loss=tf.keras.losses.MeanSquaredError(),metrics=['accuracy'])

In [201]:
model2d.fit(X_ab_Img_train,S_ab_Img_train,epochs=200,batch_size=32,validation_data=(X_ab_Img_val,S_ab_Img_val))

Epoch 1/200
69/69 [==============================] - 1s 9ms/step - loss: 0.2240 - accuracy: 0.0141 - val_loss: 0.0577 - val_accuracy: 0.0077
Epoch 2/200
69/69 [==============================] - 0s 7ms/step - loss: 0.0890 - accuracy: 0.0618 - val_loss: 0.0553 - val_accuracy: 0.0309
Epoch 3/200
69/69 [==============================] - 0s 7ms/step - loss: 0.0703 - accuracy: 0.0900 - val_loss: 0.0530 - val_accuracy: 0.0347
Epoch 4/200
69/69 [==============================] - 0s 7ms/step - loss: 0.0572 - accuracy: 0.1109 - val_loss: 0.0506 - val_accuracy: 0.0386
Epoch 5/200
69/69 [==============================] - 0s 7ms/step - loss: 0.0484 - accuracy: 0.1336 - val_loss: 0.0472 - val_accuracy: 0.0463
Epoch 6/200
69/69 [==============================] - 0s 7ms/step - loss: 0.0422 - accuracy: 0.1564 - val_loss: 0.0446 - val_accuracy: 0.0695
Epoch 7/200
69/69 [==============================] - 0s 7ms/step - loss: 0.0369 - accuracy: 0.1836 - val_loss: 0.0437 - val_accuracy: 0.0695
Epoch 8/200
6

**Testing the output of the CNN 2D model on test set 1**

In [205]:
s, sr = librosa.load('test_x_01.wav',sr=None)
Xtest = librosa.stft(s, n_fft=1024, hop_length=512)
Xtest_ab = np.abs(Xtest)
Xtest_ab_T = Xtest_ab.T
Xtest_ab_T.shape

(142, 513)

In [206]:
Xtest_ab_T = img_build(Xtest_ab_T)
Xtest_ab_T.shape

(142, 20, 513)

In [207]:
Xtest_ab_T = Xtest_ab_T.reshape(142,20,513,1)
Xtest_ab_T.shape

(142, 20, 513, 1)

In [208]:
Stest_ab = model2d.predict(Xtest_ab_T)
Stest_ab.shape

(142, 513)

In [209]:
Stest_ab = Stest_ab.T
Stest_ab.shape

(513, 142)

In [210]:
Stest = geek.multiply(np.divide(Xtest,Xtest_ab), Stest_ab)
Stest_inv = librosa.istft(Stest,hop_length=512)
Stest_inv.shape

(72192,)

In [211]:
sf.write('test_s_01_recons.wav',Stest_inv,sr)

In [430]:
sound_file = 'test_s_01_recons-CNN_2D.wav'
Audio(sound_file, autoplay=True)

**Testing the output of the CNN 2D model on test set 2**

In [212]:
s, sr = librosa.load('test_x_02.wav',sr=None)
Xtest = librosa.stft(s, n_fft=1024, hop_length=512)
Xtest_ab = np.abs(Xtest)
Xtest_ab.shape

(513, 380)

In [213]:
Xtest_ab_T = img_build(Xtest_ab.T)
Xtest_ab_T .shape

(380, 20, 513)

In [214]:
Xtest_ab_T = Xtest_ab_T.reshape(380,20,513,1)
Stest_ab =  model2d.predict(Xtest_ab_T)
Stest_ab.shape

(380, 513)

In [215]:
Stest_ab = Stest_ab.T
Stest_ab.shape

(513, 380)

In [216]:
Stest = geek.multiply(np.divide(Xtest,Xtest_ab), Stest_ab) 
Stest_inv = librosa.istft(Stest,hop_length=512)
Stest_inv.shape

(194048,)

In [217]:
sf.write('test_s_02_recons.wav',Stest_inv,sr)

In [431]:
sound_file = 'test_s_02_recons-CNN_2D.wav'
Audio(sound_file, autoplay=True)

**Checking the Signal to Noise Ratio for CNN 2D model**

In [218]:
S_ab_T_pred = model2d.predict(X_ab_Img_val)

In [203]:
#SNR Function
def SNR_func(y,ypred):
  num = np.sum(np.square(y))
  dem = np.sum(np.square(y-ypred))
  return 10 * np.log10(num/dem)

In [410]:
SNR_func(S_ab_Img_val,S_ab_T_pred)

12.3245089754532


In [411]:
%%shell
jupyter nbconvert --to html /content/DeepLearning_Assignment2_Ajinkya.ipynb

[NbConvertApp] Converting notebook /content/DeepLearning_Assignment2_Ajinkya.ipynb to html
[NbConvertApp] Writing 467432 bytes to /content/DeepLearning_Assignment2_Ajinkya.html
